In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
housePath = "data/Houses.txt"
consumptionPath = "data/MonthlyWaterConsumption.txt"

outputPath1 = "outPart1/"
outputPath2 = "outPart2/"

In [ ]:
# Define the dataframes associated with the input files
houseDF = spark.read.format("csv")\
            .option("header", False)\
            .option("inferSchema", True)\
            .load(housePath)\
            .withColumnRenamed("_c0", "HID")\
            .withColumnRenamed("_c1", "City")\
            .withColumnRenamed("_c2", "Country")\
            .withColumnRenamed("_c3", "YearBuilt")


# Register the temporary table houses 
houseDF.createOrReplaceTempView("houses")


consumptionDF = spark.read.format("csv")\
                .option("header", False)\
                .option("inferSchema", True)\
                .load(consumptionPath)\
                .withColumnRenamed("_c0", "HID")\
                .withColumnRenamed("_c1", "Month")\
                .withColumnRenamed("_c2", "M3")

# Register the temporary table houses 
consumptionDF.createOrReplaceTempView("consumption")

In [ ]:
#########################################
# PART 1
#########################################

In [ ]:
# Register a function that given a month returns the trimester
spark.udf.register("TRIMESTER",\
                   lambda month: (int(month.split("/")[1])-1)//3,\
                   IntegerType())

In [ ]:
# Register a function that given a month returns the year
spark.udf.register("EXTRACTYEAR",\
                   lambda month: int(month.split("/")[0]),\
                   IntegerType())

In [ ]:
# Select the water consumptions only for years 2022 and 2021
# Computere the water consumption for each trimester of the years 2022 and 2021
waterConsPerTrimDF = spark.sql("""SELECT HID, TRIMESTER(Month) AS Trim, EXTRACTYEAR(Month) AS Year, SUM(M3) AS TrimCons """
                               + """FROM consumption """
                               + """WHERE EXTRACTYEAR(Month)=2021 OR EXTRACTYEAR(Month)=2022 """
                               + """GROUP BY HID, TRIMESTER(Month), EXTRACTYEAR(Month)""")

# Register the temporary table waterConsPerTrim 
waterConsPerTrimDF.createOrReplaceTempView("waterConsPerTrim")

In [ ]:
# Register a function that given a year returns -1 if ther year is 2021. +1 otherwise.
def func2021(year):
    if (year==2021):
        return -1
    else:
        return +1
    

spark.udf.register("CONSYEAR",\
                func2021,\
                IntegerType())

In [ ]:
# Select the trimesters for each house with a consumption in the year 2022
# greater than the consumption in the year 2021
housesComparisonTrimConsDF = spark.sql("""SELECT HID, Trim, SUM(CONSYEAR(Year)*TrimCons) """
                                       + """FROM waterConsPerTrim """
                                       + """GROUP BY HID, Trim """
                                       + """HAVING SUM(CONSYEAR(Year)*TrimCons)>0""")


# Register the temporary table housesComparisonTrimCons 
housesComparisonTrimConsDF.createOrReplaceTempView("housesComparisonTrimCons")

In [ ]:
# Select the houses, and theirs cities, if they have an increasing consumption
# in at least 3 trimesters
housesWithIncreasingConsDF = spark.sql("""SELECT Houses.HID, City """
                                       + """FROM housesComparisonTrimCons, Houses """
                                       + """WHERE housesComparisonTrimCons.HID=Houses.HID """
                                       + """GROUP BY Houses.HID, City """
                                       + """HAVING COUNT(*)>=3""") 

In [ ]:
housesWithIncreasingConsDF.write\
        .format("csv")\
        .option("header", False)\
        .save(outputPath1)

In [ ]:
#########################################
# PART 2
#########################################

In [ ]:
# Compute the annual consumption for each house in each year
waterConsPerYearAndHouseDF = spark.sql("""SELECT HID, EXTRACTYEAR(Month) AS Year, SUM(M3) AS AnnualCons """
                                       + """FROM consumption """
                                       + """GROUP BY HID, EXTRACTYEAR(Month)""")
            

# Register the temporary table waterConsPerYearAndHouse 
waterConsPerYearAndHouseDF.createOrReplaceTempView("waterConsPerYearAndHouse")

In [ ]:
# Define the "elements of the windows" of two consecutive years for each house
# Given a tuple HID, Year, AnnualConsumption 
# the query returns two tuples:
# - HID,   Year,  AnnualConsumption  <- Second element/tuple of the window ending at year "Year" for house HID 
# - HID, Year+1, -AnnualConsumption  <- First element/tuple of the window ending at year "Year+1" for house HID 
#
# UNION ALL is needed to avoid removing duplicated tuples
elementsWindowsDF = spark.sql("""SELECT HID, Year, AnnualCons """
                              + """FROM waterConsPerYearAndHouse """
                              + """UNION ALL """
                              + """SELECT HID, Year+1 AS Year, -1*AnnualCons AS AnnualCons """
                              + """FROM waterConsPerYearAndHouse """)

# Register the temporary table elementsWindows 
elementsWindowsDF.createOrReplaceTempView("elementsWindows")

In [ ]:
# Select the pairs (HID,Year) associated with windows with a decreasing consumption 
# (if the sum of the consumptions in a window is negative it means that the consumption
#  in the first year of the window is greater than the consumption in the second year 
#  of the window). Consider only complete windows (i.e., windows with 2 elements).
#
# Each HID can be associated with many windows with a decreasing trend. 
# Distint HIDs must be selected to compute the correct result.
housesWithAtLeastOneDecreaseDF = spark.sql("""SELECT DISTINCT HID """
                                           + """FROM (SELECT HID, Year """
                                           + """      FROM elementsWindows """
                                           + """      GROUP BY HID, Year """
                                           + """      HAVING COUNT(*)=2 AND SUM(AnnualCons)<0)""")
            
            
# Register the temporary table housesWithAtLeastOneDecrease 
housesWithAtLeastOneDecreaseDF.createOrReplaceTempView("housesWithAtLeastOneDecrease")

In [ ]:
# Select the cities with at most two houses with at least one annual decrease 
# removing from all cities those with a number of houses with at least one 
# annual decrease greater than two. This approach also selects the cities with 
# 0 houses with at least one annual decrease.
res2DF = spark.sql("""SELECT DISTINCT City """
                   + """FROM Houses """
                   + """WHERE City NOT IN (SELECT City """
                   + """                   FROM Houses, housesWithAtLeastOneDecrease """
                   + """                   WHERE Houses.HID=housesWithAtLeastOneDecrease.HID """
                   + """                   GROUP BY City """
                   + """                   HAVING COUNT(*)>2)""")

In [ ]:
res2DF.write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath2)